In [1]:
from src import MachadoLoader as ml

import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell

import numpy as np

In [2]:
data = ml.readData()

In [3]:
mapedData = ml.mapData(data)
trainData,testData = ml.separateData(mapedData)

In [ ]:
trainData = [{'x':text[0:-1],'y':text[1:len(text)]} for text in trainData]
testData = [{'x':text[0:-1],'y':text[1:len(text)]} for text in testData]

In [ ]:
tf.flags.DEFINE_string("savePath","/tmp/","path to model")

In [ ]:
FLAGS = tf.flags.FLAGS

In [ ]:
def dataType():
    return tf.float32

def create_input_fn(Data,batchSize=50):
    data = Data

    def input_fn():
        np.random.shuffle(data)
        batch = 0
        batches = []
        for _ in range(int(len(data)/batchSize)):
            x = [d['x'] for d in data[batch:min(len(data),batch+batchSize)]]
            y = [d['y'] for d in data[batch:min(len(data),batch+batchSize)]]
            batch = batch+batchSize
            batches.append((x,y))
            print("new batch")
        batches = tf.constant(batches)
        return batches
    
    return input_fn

In [ ]:
b = create_input_fn(trainData)

In [ ]:
class RNN:
    def __init__(self,layers=2,rnn_size=128,n_chunks=28,chunk_size=28):
        self.layers = layers
        self.rnn_size = rnn_size
        self.n_chunks = n_chunks
        self.chunk_size = chunk_size
        
        self.sess = tf.Session()
    
    def model(self,x,n_classes):
        layer = {'weights':tf.Variable(tf.random_normal([self.rnn_size,n_classes])),
                 'biases':tf.Variable(tf.random_normal([n_classes]))}
    
        x = tf.transpose(x,[1,0,2])
        x = tf.reshape(x,[-1,self.chunk_size])
        x = tf.split(0,self.n_chunks,x)

        lstm_cell = rnn_cell.BasicLSTMCell(self.rnn_size)
        stacked_lstm = rnn_cell.MultiRNNCell([lstm_cell] * 2, state_is_tuple=True)

        outputs, states = rnn.rnn(stacked_lstm, x, dtype=tf.float32)


        output = tf.matmul(outputs[-1],layer['weights']) + layer['biases']

        return output
    
    def train(self,batch_gen, hm_epochs=3):
        prediction = self.model(tf.placeholder('float', [None, self.n_chunks,self.chunk_size]),10)
        cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(prediction,y) )
        optimizer = tf.train.AdamOptimizer().minimize(cost)

        self.sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for (epoch_x, epoch_y) in batch_gen():
                epoch_x = epoch_x.reshape((len(epoch_x),n_chunks,chunk_size))
                _, c = self.sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)
        
        self.prediction = prediction
        
    def score(dataX,dataY):
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:dataX.reshape((-1, n_chunks, chunk_size)), y:dataY}))

In [ ]:
r = RNN()
r.train(b)